In [ ]:
# Testing Cell
from aviary.mission.flight_phase_builder import FlightPhaseOptions
from aviary.mission.height_energy.phases.groundroll_phase import GroundrollPhaseOptions as FGopt
from aviary.mission.two_dof.phases.accel_phase import AccelPhaseOptions
from aviary.mission.two_dof.phases.ascent_phase import AscentPhaseOptions
from aviary.mission.two_dof.phases.cruise_phase import CruisePhaseOptions
from aviary.mission.two_dof.phases.flight_phase import FlightPhaseOptions
from aviary.mission.two_dof.phases.groundroll_phase import GroundrollPhaseOptions as GGopt
from aviary.mission.two_dof.phases.rotation_phase import RotationPhaseOptions
from aviary.mission.solved_two_dof.phases.solved_twodof_phase import SolvedTwoDOFPhaseOptions
from aviary.utils.doctape import glue_keys

dummy_phase_info = {}
dummy_phase_info.update(AccelPhaseOptions().items())
dummy_phase_info.update(AscentPhaseOptions().items())
dummy_phase_info.update(CruisePhaseOptions().items())
dummy_phase_info.update(FlightPhaseOptions().items())
dummy_phase_info.update(FGopt().items())
dummy_phase_info.update(FlightPhaseOptions().items())
dummy_phase_info.update(GGopt().items())
dummy_phase_info.update(RotationPhaseOptions().items())
dummy_phase_info.update(SolvedTwoDOFPhaseOptions().items())
glue_keys(dummy_phase_info)

# Converting from legacy `phase_info` user options

If you are using a version of Aviary that was released before 0.10.0, then you may have some aircraft phase_info files that were written before a major reorganization of the options. You will need to convert them to the new format, and this document is meant to provide some guidance. The redesign of the phase options was driven by the following goals:

* Provide a set of keys that is more consistent across the different phase types.
* Remove redundancies, particularly cases where we can replace 2 keys with 1.
* Rename keys to be more understandable to the user.
* Add missing options such as certain missing scaling parameters.

This next section of this document will explain the changes in detail, followed by a phase_info example pre- and post- conversion for the height energy and two-dof equations.

## Summary of the Changes

### Naming Convention

All of the phase_info keys have been renamed so that the variable name comes first. This means that the option originally named `optimize_mach` is now named {glue:md}`mach_optimize`. This helps with readibility, and it is recommended that all "mach" options be grouped together, etc.  

### Standardize Options for States

A "state" in Aviary is a time-integration variable in the trajectory equations of motion. The new phase_info format exposes a consistent set of options for each state across all equations. Many of the options were present in the previous format, but there are some new ones that give the user more control over scaling the state to improve optimization robustness. 

### Standardize Options for Controls

A "control" in Aviary is a dynamic variable that is not a state. It can be chosen by the optimizer or set by the user. The new phase_info format exposes a consistent set of options for each control across all equations. Note that there is considerable overlap between the options available for a state and a control.

### Parametric Control Options

The options for specifying parameteric controls have changed. The old way required that `use_polynomial_control` be set to True to run all of the phase's controls into polynomial controls, which represents each control as a polynomial with order equal to the value in `polynomial_control_order`. This is a technique used to reduce the number of design variables. The new phase_info format allows a polynomial control to be specified for each control individually using a single option {glue:md}`mach_polynomial_order`. The default value is False, which means a regular full-order control. Setting this option to an integer turns the control into a polyomial control of that order.

### Standardize Options for Time

A similar refactor was applied to the time-related phase variables, so that all of these keys start with "time". Thus, `initial_bounds` has been renamed to {glue:md}`time_initial_bounds`.

### Compact Specification of Lower and Upper Bounds

All individual "_lower" and "_upper" bounds keys are now combined into a single "_bounds" key, which now contains a tuple with the lower and upper bounds. Some variables were already using "_lower" and "_upper", so this change makes the behvaior uniform across all equations.

### Streamline Phase Boundary Conditions

The old format used the options "fix_initial", "input_initial", and "constrain_final" to broadly specify the phase boundary conditions. These were confusing to users, no flexible enough for some cases, and required changes when using `include_takeoff`. This process has been partially automated in the new format. The user now has the ability to constrain specific variables at the start or end of the phase. For example, you can now individually constrain the mach number at the end of cruise using {glue:md}`mach_final` while letting the optimizer pick the final altitude by leaving {glue:md}`altitude_final` at its default value of None. Note that constraining a variable at the end of one phase, and at the start of the next phase is redundant, and can be accomplished by constraining one end or the other.

### Remove Unused Options

The option `add_initial_mass_constraint` was leftover from an experimental feature, and has been removed.

## Height Energy Example

The following phase_info is specified in the old format:

In [ ]:
old_phase_info = {
    'pre_mission': {'include_takeoff': True, 'optimize_mass': True},
    'climb': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'fix_initial': False,
            'input_initial': True,
            'optimize_mach': True,
            'optimize_altitude': True,
            'use_polynomial_control': False,
            'num_segments': 6,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.3, 'unitless'),
            'final_mach': (0.79, 'unitless'),
            'mach_bounds': ((0.1, 0.8), 'unitless'),
            'initial_altitude': (35.0, 'ft'),
            'final_altitude': (35000.0, 'ft'),
            'altitude_bounds': ((0.0, 35000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'constrain_final': False,
            'fix_duration': False,
            'initial_bounds': ((0.0, 2.0), 'min'),
            'duration_bounds': ((5.0, 50.0), 'min'),
            'no_descent': False,
            'add_initial_mass_constraint': False,
        },
        'initial_guesses': {'time': ([0, 40.0], 'min')},
    },
    'cruise': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'optimize_mach': True,
            'optimize_altitude': True,
            'polynomial_control_order': 1,
            'use_polynomial_control': True,
            'num_segments': 1,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.79, 'unitless'),
            'final_mach': (0.79, 'unitless'),
            'mach_bounds': ((0.79, 0.79), 'unitless'),
            'initial_altitude': (35000.0, 'ft'),
            'final_altitude': (35000.0, 'ft'),
            'altitude_bounds': ((35000.0, 35000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'fix_initial': False,
            'constrain_final': False,
            'fix_duration': False,
            'initial_bounds': ((64.0, 192.0), 'min'),
            'duration_bounds': ((60.0, 720.0), 'min'),
        },
        'initial_guesses': {'time': ([128, 113], 'min')},
    },
    'descent': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'optimize_mach': True,
            'optimize_altitude': True,
            'use_polynomial_control': False,
            'num_segments': 5,
            'order': 3,
            'solve_for_distance': False,
            'initial_mach': (0.79, 'unitless'),
            'final_mach': (0.3, 'unitless'),
            'mach_bounds': ((0.2, 0.8), 'unitless'),
            'initial_altitude': (35000.0, 'ft'),
            'final_altitude': (35.0, 'ft'),
            'altitude_bounds': ((0.0, 35000.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'fix_initial': False,
            'constrain_final': True,
            'fix_duration': False,
            'initial_bounds': ((120.0, 800.0), 'min'),
            'duration_bounds': ((5.0, 35.0), 'min'),
            'no_climb': True,
        },
        'initial_guesses': {'time': ([241, 30], 'min')},
    },
    'post_mission': {
        'include_landing': True,
        'constrain_range': True,
        'target_range': (3375.0, 'nmi'),
    },
}

This phase_info has been updated to the new format below:

In [ ]:
new_phase_info = {
    'pre_mission': {'include_takeoff': True, 'optimize_mass': True},
    'climb': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 6,
            'order': 3,
            'mach_bounds': ((0.1, 0.8), 'unitless'),
            'mach_optimize': True,
            'altitude_bounds': ((0.0, 35000.0), 'ft'),
            'altitude_optimize': True,
            'throttle_enforcement': 'path_constraint',
            'mass_ref': (200000, 'lbm'),
            'time_initial': (0.0, 'min'),
            'time_duration_bounds': ((20.0, 60.0), 'min'),
            'no_descent': True,
        },
        'initial_guesses': {
            'time': ([0, 40.0], 'min'),
            'altitude': ([35, 35000.0], 'ft'),
            'mach': ([0.3, 0.79], 'unitless'),
        },
    },
    'cruise': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'mach_initial': (0.79, 'unitless'),
            'mach_bounds': ((0.79, 0.79), 'unitless'),
            'mach_optimize': True,
            'mach_polynomial_order': 1,
            'altitude_initial': (35000.0, 'ft'),
            'altitude_bounds': ((35000.0, 35000.0), 'ft'),
            'altitude_optimize': True,
            'altitude_polynomial_order': 1,
            'throttle_enforcement': 'boundary_constraint',
            'mass_ref': (200000, 'lbm'),
            'time_initial_bounds': ((20.0, 60.0), 'min'),
            'time_duration_bounds': ((60.0, 720.0), 'min'),
        },
        'initial_guesses': {
            'time': ([40, 200], 'min'),
            'altitude': ([35000, 35000.0], 'ft'),
            'mach': ([0.79, 0.79], 'unitless'),
        },
    },
    'descent': {
        'subsystem_options': {'core_aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 5,
            'order': 3,
            'mach_initial': (0.79, 'unitless'),
            'mach_final': (0.3, 'unitless'),
            'mach_bounds': ((0.2, 0.8), 'unitless'),
            'mach_optimize': True,
            'altitude_initial': (35000.0, 'ft'),
            'altitude_final': (35.0, 'ft'),
            'altitude_bounds': ((0.0, 35000.0), 'ft'),
            'altitude_optimize': True,
            'throttle_enforcement': 'path_constraint',
            'mass_ref': (200000, 'lbm'),
            'distance_ref': (3375, 'nmi'),
            'time_initial_bounds': ((80.0, 780.0), 'min'),
            'time_duration_bounds': ((5.0, 45.0), 'min'),
            'no_climb': True,
        },
        'initial_guesses': {
            'time': ([240, 30], 'min'),
        },
    },
    'post_mission': {
        'include_landing': True,
        'constrain_range': True,
        'target_range': (3375.0, 'nmi'),
    },
}

In [ ]:
# Testing Cell
from aviary.interface.methods_for_level2 import AviaryProblem

prob = AviaryProblem()

prob.load_inputs('models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv', new_phase_info)

prob.check_and_preprocess_inputs()
prob.add_pre_mission_systems()
prob.add_phases()

## Two-DOF Example

The following phase_info is specified in the old format:

In [ ]:
from aviary.variable_info.enums import SpeedType

# defaults for 2DOF based phases
mission_distance = 3675

old_phase_info = {
    'groundroll': {
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'connect_initial_mass': False,
            'fix_initial': True,
            'fix_initial_mass': False,
            'duration_ref': (50.0, 's'),
            'duration_bounds': ((1.0, 100.0), 's'),
            'velocity_lower': (0, 'kn'),
            'velocity_upper': (1000, 'kn'),
            'velocity_ref': (150, 'kn'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'ft'),
            'distance_upper': (10.0e3, 'ft'),
            'distance_ref': (3000, 'ft'),
            'distance_defect_ref': (3000, 'ft'),
        },
        'initial_guesses': {
            'time': ([0.0, 40.0], 's'),
            'velocity': ([0.066, 143.1], 'kn'),
            'distance': ([0.0, 1000.0], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'rotation': {
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'fix_initial': False,
            'duration_bounds': ((1, 100), 's'),
            'duration_ref': (50.0, 's'),
            'velocity_lower': (0, 'kn'),
            'velocity_upper': (1000, 'kn'),
            'velocity_ref': (100, 'kn'),
            'velocity_ref0': (0, 'kn'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'ft'),
            'distance_upper': (10_000, 'ft'),
            'distance_ref': (5000, 'ft'),
            'distance_defect_ref': (5000, 'ft'),
            'angle_lower': (0.0, 'rad'),
            'angle_upper': (5.0, 'rad'),
            'angle_ref': (5.0, 'rad'),
            'angle_defect_ref': (5.0, 'rad'),
            'normal_ref': (10000, 'lbf'),
        },
        'initial_guesses': {
            'time': ([40.0, 5.0], 's'),
            'angle_of_attack': ([0.0, 2.5], 'deg'),
            'velocity': ([143, 150.0], 'kn'),
            'distance': ([3680.37217765, 4000], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'ascent': {
        'user_options': {
            'num_segments': 4,
            'order': 3,
            'fix_initial': False,
            'velocity_lower': (0, 'kn'),
            'velocity_upper': (700, 'kn'),
            'velocity_ref': (200, 'kn'),
            'velocity_ref0': (0, 'kn'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'ft'),
            'distance_upper': (15_000, 'ft'),
            'distance_ref': (1e4, 'ft'),
            'distance_defect_ref': (1e4, 'ft'),
            'alt_lower': (0, 'ft'),
            'alt_upper': (700, 'ft'),
            'alt_ref': (1000, 'ft'),
            'alt_defect_ref': (1000, 'ft'),
            'final_altitude': (500, 'ft'),
            'alt_constraint_ref': (500, 'ft'),
            'angle_lower': (-10, 'rad'),
            'angle_upper': (20, 'rad'),
            'angle_ref': (57.2958, 'deg'),
            'angle_defect_ref': (57.2958, 'deg'),
            'pitch_constraint_lower': (0.0, 'deg'),
            'pitch_constraint_upper': (15.0, 'deg'),
            'pitch_constraint_ref': (1.0, 'deg'),
        },
        'initial_guesses': {
            'time': ([45.0, 25.0], 's'),
            'flight_path_angle': ([0.0, 8.0], 'deg'),
            'angle_of_attack': ([2.5, 1.5], 'deg'),
            'velocity': ([150.0, 185.0], 'kn'),
            'distance': ([4.0e3, 10.0e3], 'ft'),
            'altitude': ([0.0, 500.0], 'ft'),
            'tau_gear': (0.2, 'unitless'),
            'tau_flaps': (0.9, 'unitless'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'accel': {
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'fix_initial': False,
            'alt': (500, 'ft'),
            'EAS_constraint_eq': (250, 'kn'),
            'duration_bounds': ((1, 200), 's'),
            'duration_ref': (1000, 's'),
            'velocity_lower': (150, 'kn'),
            'velocity_upper': (270, 'kn'),
            'velocity_ref': (250, 'kn'),
            'velocity_ref0': (150, 'kn'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'NM'),
            'distance_upper': (150, 'NM'),
            'distance_ref': (5, 'NM'),
            'distance_defect_ref': (5, 'NM'),
        },
        'initial_guesses': {
            'time': ([70.0, 13.0], 's'),
            'velocity': ([185.0, 250.0], 'kn'),
            'distance': ([10.0e3, 20.0e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'climb1': {
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'fix_initial': False,
            'EAS_target': (250, 'kn'),
            'mach_cruise': 0.8,
            'target_mach': False,
            'final_altitude': (10.0e3, 'ft'),
            'duration_bounds': ((30, 300), 's'),
            'duration_ref': (1000, 's'),
            'alt_lower': (400, 'ft'),
            'alt_upper': (11_000, 'ft'),
            'alt_ref': (10.0e3, 'ft'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'NM'),
            'distance_upper': (500, 'NM'),
            'distance_ref': (10, 'NM'),
            'distance_ref0': (0, 'NM'),
        },
        'initial_guesses': {
            'time': ([1.0, 2.0], 'min'),
            'distance': ([20.0e3, 100.0e3], 'ft'),
            'altitude': ([500.0, 10.0e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'climb2': {
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'fix_initial': False,
            'EAS_target': (270, 'kn'),
            'mach_cruise': 0.8,
            'target_mach': True,
            'final_altitude': (37.5e3, 'ft'),
            'required_available_climb_rate': (0.1, 'ft/min'),
            'duration_bounds': ((200, 17_000), 's'),
            'duration_ref': (5000, 's'),
            'alt_lower': (9000, 'ft'),
            'alt_upper': (40000, 'ft'),
            'alt_ref': (30000, 'ft'),
            'alt_ref0': (0, 'ft'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (10, 'NM'),
            'distance_upper': (1000, 'NM'),
            'distance_ref': (500, 'NM'),
            'distance_ref0': (0, 'NM'),
            'distance_defect_ref': (500, 'NM'),
        },
        'initial_guesses': {
            'time': ([216.0, 1300.0], 's'),
            'distance': ([100.0e3, 200.0e3], 'ft'),
            'altitude': ([10.0e3, 37.5e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'cruise': {
        'user_options': {
            'alt_cruise': (37.5e3, 'ft'),
            'mach_cruise': 0.8,
        },
        'initial_guesses': {
            # [Initial mass, delta mass] for special cruise phase.
            'mass': ([171481.0, -35000], 'lbm'),
            'initial_distance': (200.0e3, 'ft'),
            'initial_time': (1516.0, 's'),
            'altitude': (37.5e3, 'ft'),
            'mach': (0.8, 'unitless'),
        },
    },
    'desc1': {
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'fix_initial': False,
            'input_initial': False,
            'EAS_limit': (350, 'kn'),
            'mach_cruise': 0.8,
            'input_speed_type': SpeedType.MACH,
            'final_altitude': (10.0e3, 'ft'),
            'duration_bounds': ((300.0, 900.0), 's'),
            'duration_ref': (1000, 's'),
            'alt_lower': (1000, 'ft'),
            'alt_upper': (40_000, 'ft'),
            'alt_ref': (30_000, 'ft'),
            'alt_ref0': (0, 'ft'),
            'alt_constraint_ref': (10000, 'ft'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (140_000, 'lbm'),
            'mass_ref0': (0, 'lbm'),
            'mass_defect_ref': (140_000, 'lbm'),
            'distance_lower': (0.0, 'NM'),
            'distance_upper': (5000.0, 'NM'),
            'distance_ref': (mission_distance, 'NM'),
            'distance_ref0': (0, 'NM'),
            'distance_defect_ref': (100, 'NM'),
        },
        'initial_guesses': {
            'mass': (136000.0, 'lbm'),
            'altitude': ([37.5e3, 10.0e3], 'ft'),
            'throttle': ([0.0, 0.0], 'unitless'),
            'distance': ([0.92 * mission_distance, 0.96 * mission_distance], 'NM'),
            'time': ([28000.0, 500.0], 's'),
        },
    },
    'desc2': {
        'user_options': {
            'num_segments': 1,
            'order': 7,
            'fix_initial': False,
            'input_initial': False,
            'EAS_limit': (250, 'kn'),
            'mach_cruise': 0.80,
            'input_speed_type': SpeedType.EAS,
            'final_altitude': (1000, 'ft'),
            'duration_bounds': ((100.0, 5000), 's'),
            'duration_ref': (500, 's'),
            'alt_lower': (500, 'ft'),
            'alt_upper': (11_000, 'ft'),
            'alt_ref': (10.0e3, 'ft'),
            'alt_ref0': (1000, 'ft'),
            'alt_constraint_ref': (1000, 'ft'),
            'mass_lower': (0, 'lbm'),
            'mass_upper': (None, 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_lower': (0, 'NM'),
            'distance_upper': (5000, 'NM'),
            'distance_ref': (3500, 'NM'),
            'distance_defect_ref': (100, 'NM'),
        },
        'initial_guesses': {
            'mass': (136000.0, 'lbm'),
            'altitude': ([10.0e3, 1.0e3], 'ft'),
            'throttle': ([0.0, 0.0], 'unitless'),
            'distance': ([0.96 * mission_distance, mission_distance], 'NM'),
            'time': ([28500.0, 500.0], 's'),
        },
    },
}

This phase_info has been updated to the new format below:

In [ ]:
new_phase_info = {
    'groundroll': {
        'subsystem_options': {'core_aerodynamics': {'method': 'low_speed'}},
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'time_initial': (0.0, 's'),
            'time_duration_ref': (50.0, 's'),
            'time_duration_bounds': ((1.0, 100.0), 's'),
            'velocity_initial': (0.066, 'kn'),
            'velocity_bounds': ((0, 1000), 'kn'),
            'velocity_ref': (150, 'kn'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_initial': (0.0, 'ft'),
            'distance_bounds': ((0, 10.0e3), 'ft'),
            'distance_ref': (3000, 'ft'),
            'distance_defect_ref': (3000, 'ft'),
        },
        'initial_guesses': {
            'time': ([0.0, 40.0], 's'),
            'velocity': ([0.066, 143.1], 'kn'),
            'distance': ([0.0, 1000.0], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'rotation': {
        'subsystem_options': {'core_aerodynamics': {'method': 'low_speed'}},
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'time_duration_bounds': ((1, 100), 's'),
            'time_duration_ref': (50.0, 's'),
            'velocity_bounds': ((0, 1000), 'kn'),
            'velocity_ref': (150, 'kn'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((0, 10.0e3), 'ft'),
            'distance_ref': (5000, 'ft'),
            'distance_defect_ref': (5000, 'ft'),
            'angle_of_attack_initial': (0.0, 'deg'),
            'angle_of_attack_bounds': ((0.0, 12.0), 'deg'),
            'angle_of_attack_ref': (12.0, 'deg'),
            'angle_of_attack_defect_ref': (12.0, 'deg'),
            'normal_ref': (10000, 'lbf'),
        },
        'initial_guesses': {
            'time': ([40.0, 5.0], 's'),
            'angle_of_attack': ([0.0, 2.5], 'deg'),
            'velocity': ([143, 150.0], 'kn'),
            'distance': ([3680.37217765, 4000], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'ascent': {
        'subsystem_options': {'core_aerodynamics': {'method': 'low_speed'}},
        'user_options': {
            'num_segments': 4,
            'order': 3,
            'velocity_bounds': ((0, 700), 'kn'),
            'velocity_ref': (200, 'kn'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((0, 15_000), 'ft'),
            'distance_ref': (1e4, 'ft'),
            'distance_defect_ref': (1e4, 'ft'),
            'altitude_bounds': ((0.0, 700.0), 'ft'),
            'altitude_ref': (1000, 'ft'),
            'altitude_defect_ref': (1000, 'ft'),
            'altitude_initial': (0, 'ft'),
            'altitude_final': (500, 'ft'),
            'altitude_constraint_ref': (500, 'ft'),
            'flight_path_angle_bounds': ((-10.0, 20.0), 'deg'),
            'flight_path_angle_ref': (57.2958, 'deg'),
            'flight_path_angle_defect_ref': (57.2958, 'deg'),
            'flight_path_angle_initial': (0.0, 'deg'),
            'pitch_constraint_bounds': ((0.0, 15.0), 'deg'),
            'pitch_constraint_ref': (1.0, 'deg'),
        },
        'initial_guesses': {
            'time': ([45.0, 25.0], 's'),
            'flight_path_angle': ([0.0, 8.0], 'deg'),
            'angle_of_attack': ([2.5, 1.5], 'deg'),
            'velocity': ([150.0, 185.0], 'kn'),
            'distance': ([4.0e3, 10.0e3], 'ft'),
            'altitude': ([0.0, 500.0], 'ft'),
            'tau_gear': (0.2, 'unitless'),
            'tau_flaps': (0.9, 'unitless'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'accel': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'alt': (500, 'ft'),
            'EAS_constraint_eq': (250, 'kn'),
            'time_duration_bounds': ((1, 200), 's'),
            'time_duration_ref': (1000, 's'),
            'velocity_bounds': ((150, 270), 'kn'),
            'velocity_ref': (250, 'kn'),
            'velocity_ref0': (150, 'kn'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((0, 150.0), 'NM'),
            'distance_ref': (5, 'NM'),
            'distance_defect_ref': (5, 'NM'),
        },
        'initial_guesses': {
            'time': ([70.0, 13.0], 's'),
            'velocity': ([185.0, 250.0], 'kn'),
            'distance': ([10.0e3, 20.0e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'climb1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'num_segments': 1,
            'order': 3,
            'EAS_target': (250, 'kn'),
            'mach_target': 0.8,
            'time_duration_bounds': ((30, 300), 's'),
            'time_duration_ref': (1000, 's'),
            'altitude_initial': (500.0, 'ft'),
            'altitude_final': (10.0e3, 'ft'),
            'altitude_bounds': ((400.0, 11_000.0), 'ft'),
            'altitude_ref': (10.0e3, 'ft'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((0, 500.0), 'NM'),
            'distance_ref': (10, 'NM'),
        },
        'initial_guesses': {
            'time': ([1.0, 2.0], 'min'),
            'distance': ([20.0e3, 100.0e3], 'ft'),
            'altitude': ([500.0, 10.0e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'climb2': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'EAS_target': (270, 'kn'),
            'mach_target': 0.8,
            'required_available_climb_rate': (0.1, 'ft/min'),
            'time_duration_bounds': ((200, 17_000), 's'),
            'time_duration_ref': (5000, 's'),
            'altitude_final': (37.5e3, 'ft'),
            'altitude_bounds': ((9000.0, 40000.0), 'ft'),
            'altitude_ref': (30000, 'ft'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((10.0, 1000.0), 'NM'),
            'distance_ref': (500, 'NM'),
            'distance_defect_ref': (500, 'NM'),
        },
        'initial_guesses': {
            'time': ([216.0, 1300.0], 's'),
            'distance': ([100.0e3, 200.0e3], 'ft'),
            'altitude': ([10.0e3, 37.5e3], 'ft'),
            'throttle': ([0.956, 0.956], 'unitless'),
        },
    },
    'cruise': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'alt_cruise': (37.5e3, 'ft'),
            'mach_cruise': 0.8,
        },
        'initial_guesses': {
            # [Initial mass, delta mass] for special cruise phase.
            'mass': ([171481.0, -35000], 'lbm'),
            'initial_distance': (200.0e3, 'ft'),
            'initial_time': (1516.0, 's'),
            'altitude': (37.5e3, 'ft'),
            'mach': (0.8, 'unitless'),
        },
    },
    'desc1': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'EAS_target': (350, 'kn'),
            'mach_target': 0.8,
            'input_speed_type': SpeedType.MACH,
            'time_duration_bounds': ((300.0, 900.0), 's'),
            'time_duration_ref': (1000, 's'),
            'altitude_final': (10.0e3, 'ft'),
            'altitude_bounds': ((1000.0, 40_000.0), 'ft'),
            'altitude_ref': (30_000, 'ft'),
            'altitude_constraint_ref': (10000, 'ft'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (140_000, 'lbm'),
            'mass_defect_ref': (140_000, 'lbm'),
            'distance_bounds': ((0.0, 5000.0), 'NM'),
            'distance_ref': (mission_distance, 'NM'),
            'distance_defect_ref': (100, 'NM'),
        },
        'initial_guesses': {
            'mass': (136000.0, 'lbm'),
            'altitude': ([37.5e3, 10.0e3], 'ft'),
            'throttle': ([0.0, 0.0], 'unitless'),
            'distance': ([0.92 * mission_distance, 0.96 * mission_distance], 'NM'),
            'time': ([28000.0, 500.0], 's'),
        },
    },
    'desc2': {
        'subsystem_options': {'core_aerodynamics': {'method': 'cruise'}},
        'user_options': {
            'num_segments': 1,
            'order': 7,
            'EAS_target': (250, 'kn'),
            'mach_target': 0.80,
            'input_speed_type': SpeedType.EAS,
            'time_duration_bounds': ((100.0, 5000), 's'),
            'time_duration_ref': (500, 's'),
            'altitude_final': (1000, 'ft'),
            'altitude_bounds': ((500.0, 11_000.0), 'ft'),
            'altitude_ref': (10.0e3, 'ft'),
            'altitude_ref0': (1000, 'ft'),
            'altitude_constraint_ref': (1000, 'ft'),
            'mass_bounds': ((0, None), 'lbm'),
            'mass_ref': (150_000, 'lbm'),
            'mass_defect_ref': (150_000, 'lbm'),
            'distance_bounds': ((0.0, 5000.0), 'NM'),
            'distance_ref': (3500, 'NM'),
            'distance_defect_ref': (100, 'NM'),
        },
        'initial_guesses': {
            'mass': (136000.0, 'lbm'),
            'altitude': ([10.0e3, 1.0e3], 'ft'),
            'throttle': ([0.0, 0.0], 'unitless'),
            'distance': ([0.96 * mission_distance, mission_distance], 'NM'),
            'time': ([28500.0, 500.0], 's'),
        },
    },
}

In [ ]:
# Testing Cell

prob = AviaryProblem()

prob.load_inputs('models/aircraft/test_aircraft/aircraft_for_bench_GwGm.csv', new_phase_info)

prob.check_and_preprocess_inputs()
prob.add_pre_mission_systems()
prob.add_phases()